In [2]:
import os, pandas as pd
from sqlalchemy import create_engine, Column, MetaData, text, engine
import getpass,time
import pysftp,sys

# Q.1

## Download instown.files from course server

In [48]:
# set up sftp connection
sftp_ip='147.8.117.176'
sftp_port=22222
sftp_username='foo'
password='alanisthecoolest123'

cnopts = pysftp.CnOpts()
cnopts.hostkeys = None
sftp = pysftp.Connection(host=sftp_ip, username=sftp_username, password=password, port=sftp_port, cnopts=cnopts)

In [51]:
# download files
localpath = '../data/'
remotepath = 'upload/data/'
files = ['instown.file%s.parquet' % i for i in range(1,4)]
for file in files:
    sftp.get(remotepath + file, localpath + file)

## Running queries on course server

In [5]:
user = getpass.getpass()
password = getpass.getpass()

········
········


In [6]:
def mysql_query(query,user=user, password=password, host='jarvis.workisboring.com', port=9030):
    # check if port is in 9030, 19030, or 3306, 9005
    # start time
    start_time = time.time()
    
    if port in [9030, 19030, 3306,9004]:
        engine = create_engine('mysql+pymysql://'+user+':'+password+'@'+host+':'+str(port))
        #df = pd.read_sql(query, engine)
        df = pd.read_sql_query(sql=text(query), con=engine.connect())
        # end time
        end_time = time.time()
        # seconds
        elapsed_time = end_time - start_time
        print(f'Elapsed time: {elapsed_time} seconds')
        return df    
    else:
        print('port is not in 9030, 19030, or 3306, please check')
        return None

In [62]:
query = """SELECT mgrname, ABS(shares * prc) AS mktcap
           FROM instown.file1
           INNER JOIN instown.file3 AS file3 ON file3.mgrno = file1.mgrno AND
           file1.fdate = file3.fdate
           INNER JOIN instown.file2 AS file2 ON file2.fdate = file3.fdate AND
           file2.cusip = file3.cusip
           WHERE ticker = 'AAPL' AND file3.fdate = '2018-03-31'
           ORDER BY shares DESC
           LIMIT 5;"""
mysql_query(query,user=user,password=password,port=19030)

Elapsed time: 1.0593407154083252 seconds


,mgrname,mktcap
0,"VANGUARD GROUP, INC.",5.885398e+10
1,BLACKROCK INC,5.359712e+10
2,BERKSHIRE HATHAWAY INC.,4.022580e+10
3,STATE STR CORPORATION,3.311086e+10
4,FIDELITY MGMT & RESEARCH CO,1.638478e+10


## Running queries on local database: Apache Drill

In [44]:
from pydrill.client import PyDrill
drill = PyDrill(host='localhost', port=8047)

if not drill.is_active():
    raise ImproperlyConfigured('Please run Drill first')

In [47]:
query = """SELECT file1.mgrname, ABS(file3.shares * file2.prc) AS mktcap
           FROM myDfs.MFIN7035.`instown.file1.parquet` as file1
           INNER JOIN myDfs.MFIN7035.`instown.file3.parquet` AS file3 ON file3.mgrno = file1.mgrno AND
           file1.fdate = file3.fdate
           INNER JOIN myDfs.MFIN7035.`instown.file2.parquet` AS file2 ON file2.fdate = file3.fdate AND
           file2.cusip = file3.cusip
           WHERE file2.ticker = 'AAPL' AND file3.fdate = '2018-03-31'
           ORDER BY file3.shares DESC
           LIMIT 5"""
sta = time.time()
data = drill.query(query, timeout = 90)
end = time.time()
print("Elapsed time: %s seconds" % (end - sta))
data.to_dataframe()

Elapsed time: 34.9938178062439 seconds


,mgrname,mktcap
0,"VANGUARD GROUP, INC.",5.885398e+10
1,BLACKROCK INC,5.359712e+10
2,BERKSHIRE HATHAWAY INC.,4.022580e+10
3,STATE STR CORPORATION,3.311086e+10
4,FIDELITY MGMT & RESEARCH CO,1.638478e+10


# Q.2

## Run the following codes on Heidi SQL interface.

/* OALP */
/* Neglect instown.file4 and further reduce date range to prevent RAM running out */

CREATE TABLE student_lin_jianan_595.join_instowns as
select
    *
from
    (SELECT * from instown.file2 as file2
    WHERE fdate > DATE('2021-10-01')) a
    inner join 
    (SELECT * from instown.file3 as file3
    inner join instown.file1 as file1
    USING (fdate,mgrno)
    WHERE fdate > DATE('2021-10-01')) b
    USING (fdate, cusip)

/* OLTP */

CREATE TABLE student_lin_jianan_595.instown_file1 as
select
    *
FROM
    instown.file1
WHERE fdate > DATE('2021-10-01');

CREATE TABLE student_lin_jianan_595.instown_file2 as
select
    *
FROM
    instown.file2
WHERE fdate > DATE('2021-10-01');

CREATE TABLE student_lin_jianan_595.instown_file3 as
select
    *
FROM
    instown.file3
WHERE fdate > DATE('2021-10-01');

# Q.3

## Data quality questions

### Too many missing values

In [9]:
query = """ SELECT COUNT(*) as missing_values FROM crsp.dsenames WHERE shrcls IS NULL; """
mysql_query(query,user=user,password=password,port=3306)

Elapsed time: 0.2491302490234375 seconds


,missing_values
0,104480


In [10]:
query = """ SELECT COUNT(*) as total_rows FROM crsp.dsenames; """
mysql_query(query,user=user,password=password,port=3306)

Elapsed time: 0.16151213645935059 seconds


,total_rows
0,111652


### Duplicated rows

In [16]:
query = """ SELECT
                *
            FROM
                (SELECT 
                    mgrname, COUNT(fdate) AS `dates`, COUNT(DISTINCT fdate) AS distinct_dates 
                FROM instown.file1 
                GROUP BY mgrname) a
            WHERE a.`dates` != a.distinct_dates
            limit 5; """
mysql_query(query,user=user,password=password,port=3306)

Elapsed time: 1.3332998752593994 seconds


,mgrname,dates,distinct_dates
0,1832 ASSET MANAGEMENT L.P.,36,34
1,"ADVISORY ALPHA, LLC",48,32
2,ALLIANZ GBL INVESTORS EURO GMB,9,6
3,ALTUS CAPITAL INC.,20,19
4,AMERICAN EXPRESS ASSET MANAGEM,4,2


### Unreasonable values

In [5]:
query = """ SELECT * FROM hw.hw1_stock_characteristics WHERE mcap=0 LIMIT 10; """
mysql_query(query,user=user,password=password,port=3306)

Elapsed time: 0.2038736343383789 seconds


,permno,year,volatility,mcap,beta_mktrf
0,75133,1989,0.039129,0.0,2.821369
1,43289,1970,0.028458,0.0,1.089043
2,75123,1990,0.045374,0.0,2.767916
3,75027,1992,0.012075,0.0,0.353997
4,75120,1992,0.244805,0.0,2.707153
5,68259,1988,0.039551,0.0,2.898426
6,75127,1989,0.041702,0.0,2.597417
7,75125,1992,0.022066,0.0,1.165932
8,68240,1988,0.004853,0.0,0.148206
9,75016,1992,0.184911,0.0,2.243973
